# Vision models examples
cifar10_cnn.py: Trains a simple deep CNN on the CIFAR10 small images dataset.

Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import os
print(keras.__version__)

Using TensorFlow backend.


2.1.0


In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

The following step consumes much time.

The solution is: 
- first download the cifar-10-python.tar.gz by myself,
- move the archive file into directory: ./keras/datasets/
- rename the file into cifar-10-batches-py.tar.gz. This step is very important, because the codes will detect this name.
- tar xvfz cifar-10-python.tar.gz

In [3]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
y_test.shape

(10000, 1, 10)

The original y_train and y_test is of shape (None, 1, 10),

However, the fit function requires shape (None, 10),

So here reshape the target array.

In [8]:
y_train = y_train.reshape(50000,10)
y_test = y_test.reshape(10000,10)

In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.output_shape

(None, 32, 32, 32)

In [10]:
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.output_shape

(None, 30, 30, 32)

In [11]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.output_shape

(None, 15, 15, 32)

In [12]:
model.add(Dropout(0.25))
model.output_shape

(None, 15, 15, 32)

In [13]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
print(model.output_shape)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
print(model.output_shape)

(None, 15, 15, 64)
(None, 13, 13, 64)
(None, 6, 6, 64)


In [14]:
model.add(Flatten())
print(model.output_shape)
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
print(model.output_shape)
model.add(Dense(num_classes))
model.add(Activation('softmax'))
print(model.output_shape)

(None, 2304)
(None, 512)
(None, 10)


In [15]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [16]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [17]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

`ImageDataGenerator`: process the image data. If the number of images is not enough for training, by using this generator will have more image data.

It can 
- scale the feature or image, 
- normalize the feaure or image, 
- ZCA reduce dimensions, 
- rotate the image, 
- shift the image (along width or height),
- flip the image (along horizontal or vertical)

In [18]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=int(np.ceil(x_train.shape[0] / float(batch_size))),
                        epochs=10, # epochs=100 will take too much time, here epochs=10
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.
Epoch 1/10
1563/1563 [==============================] - 168s 107ms/step - loss: 1.8257 - acc: 0.3299 - val_loss: 1.5383 - val_acc: 0.4420
Epoch 2/10
1563/1563 [==============================] - 167s 107ms/step - loss: 1.5494 - acc: 0.4341 - val_loss: 1.3783 - val_acc: 0.5038
Epoch 3/10
1563/1563 [==============================] - 166s 107ms/step - loss: 1.4244 - acc: 0.4838 - val_loss: 1.2423 - val_acc: 0.5550
Epoch 4/10
1563/1563 [==============================] - 168s 108ms/step - loss: 1.3418 - acc: 0.5180 - val_loss: 1.1708 - val_acc: 0.5824
Epoch 5/10
1563/1563 [==============================] - 173s 111ms/step - loss: 1.2711 - acc: 0.5483 - val_loss: 1.1159 - val_acc: 0.6039
Epoch 6/10
1563/1563 [==============================] - 173s 111ms/step - loss: 1.2101 - acc: 0.5724 - val_loss: 1.0334 - val_acc: 0.6357
Epoch 7/10
1563/1563 [==============================] - 173s 111ms/step - loss: 1.1527 - acc: 0.5913 - val_loss: 0.9762 - val_acc: 0.6525

In [19]:
# Save model and weights

# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# model_path = os.path.join(save_dir, model_name)
# model.save(model_path)
# print('Saved trained model at %s ' % model_path)

In [20]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 9s 872us/step
Test loss: 0.938150563622
Test accuracy: 0.6734
